In [ ]:
import warnings
from collections import Counter
import matplotlib.pyplot as plt
import torchvision
import torchvision.models as models
from torchvision.models import ViT_B_16_Weights
from torch.utils.data import Subset, Dataset, DataLoader
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
warnings.filterwarnings('ignore')
import torch
import pickle
import torch.optim as optim
import pandas as pd
import time
import os
from PIL import Image
import torchvision.transforms as transforms
import spacy
import numpy as np
import torch.nn as nn
torch.backends.cudnn.benchmark = True

# Image preprocessing pipeline
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        return image_transform(image)
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_caption(captions, batch_size=1000):
    captions = [str(c).strip() for c in captions if not pd.isna(c)]
    results = []
    for i in range(0, len(captions), batch_size):
        batch = captions[i:i + batch_size]
        docs = nlp.pipe([caption.lower() for caption in batch], disable=['parser', 'ner'])
        for doc in docs:
            tokens = [token.text for token in doc if not token.is_punct and token.text.strip()]
            results.append(tokens)
    return results

# Dataset
class Flickr30kDataset(Dataset):
    def __init__(self, image_dir, annotation_file, image_transform=None):
        self.df = pd.read_csv(annotation_file, sep=',', names=['image', 'caption'], skiprows=1)
        self.df['caption'] = self.df['caption'].fillna('').astype(str)
        self.image_dir = image_dir
        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = row['image']
        caption = row['caption']
        img_path = os.path.join(self.image_dir, img_name)
        image = preprocess_image(img_path)
        if image is None:
            return None
        caption_tokens = preprocess_caption([caption])[0]
        return image, caption_tokens, img_name

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None, None, None
    images, captions, image_names = zip(*batch)
    images = torch.stack(images)
    return images, captions, image_names

# Initialize dataset and splits
image_dir = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\Images'
annotation_file = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\captions.txt'

dataset = Flickr30kDataset(image_dir, annotation_file, image_transform)
print("Dataset loaded, size:", len(dataset))

indices = np.arange(len(dataset))
np.random.seed(42)
np.random.shuffle(indices)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)
test_subset = Subset(dataset, test_indices)
print("Train subset size:", len(train_subset))
print("Val subset size:", len(val_subset))
print("Test subset size:", len(test_subset))

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, collate_fn=custom_collate_fn)

# Verify pipeline
def show_sample(image, caption):
    try:
        if not isinstance(image, torch.Tensor):
            raise ValueError("Image is not a torch.Tensor")
        if image.shape != (3, 224, 224):
            raise ValueError(f"Unexpected image shape: {image.shape}")
        image = image.permute(1, 2, 0).numpy()
        image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        image = np.clip(image, 0, 1)
        if not isinstance(caption, list) or not all(isinstance(word, str) for word in caption):
            raise ValueError(f"Invalid caption format: {caption}")
        caption_text = ' '.join(caption).strip()
        plt.imshow(image)
        plt.title(caption_text)
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Error in show_sample: {e}")

for batch in train_loader:
    images, captions, _ = batch
    if images is None or captions is None:
        print("Skipping batch with None values")
        continue
    print(f"Image batch shape: {images.shape}")
    print(f"Sample caption: {captions[0]}")
    show_sample(images[0], captions[0])
    break

# Load pre-trained ResNet-50
resnet = models.resnet50(pretrained=True).cuda()
resnet.eval()
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final layer

# Create directory for ResNet features
feature_dir = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\Features'
os.makedirs(feature_dir, exist_ok=True)

def extract_features(loader, split_name, subset_indices):
    features = []
    captions_list = []
    image_names = []
    for batch_idx, (images, captions, batch_image_names) in enumerate(loader):
        if images is None:
            continue
        images = images.cuda()
        with torch.no_grad():
            batch_features = resnet(images).squeeze(-1).squeeze(-1)  # [batch_size, 2048]
        features.append(batch_features.cpu())
        captions_list.extend(captions)
        image_names.extend(batch_image_names)
    features = torch.cat(features, dim=0)
    torch.save({'features': features, 'captions': captions_list, 'image_names': image_names},
               os.path.join(feature_dir, f'{split_name}_features.pt'))
    print(f"Saved {split_name} features: {features.shape}")

# Extract ResNet features
extract_features(train_loader, 'train', train_indices)
extract_features(val_loader, 'val', val_indices)
extract_features(test_loader, 'test', test_indices)

# Load pre-trained ViT
vit = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1).cuda()
vit.heads = nn.Identity()  # Remove classification head
vit.eval()

# Create directory for ViT features
vit_feature_dir = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\ViT_Features'
os.makedirs(vit_feature_dir, exist_ok=True)

def extract_vit_features(loader, split_name):
    print(f"Starting {split_name} feature extraction")
    features = []
    captions_list = []
    image_names = []
    start_time = time.time()
    with torch.amp.autocast('cuda'), torch.no_grad():
        for batch_idx, (images, captions, batch_image_names) in enumerate(tqdm(loader, desc=f"Processing {split_name}")):
            if images is None:
                print(f"Skipping batch {batch_idx + 1}: No images")
                continue
            images = images.cuda()
            batch_features = vit(images)  # [batch_size, 768]
            features.append(batch_features.cpu())
            captions_list.extend(captions)
            image_names.extend(batch_image_names)
        if features:
            final_features = torch.cat(features, dim=0)
        else:
            final_features = torch.tensor([]).reshape(0, 768)
        torchsave_path = os.path.join(vit_feature_dir, f'{split_name}_vit_features.pt')
        torch.save({
            'features': final_features,
            'captions': captions_list,
            'image_names': image_names
        }, torchsave_path)
        print(f"Saved {split_name} ViT features: {final_features.shape}")
        print(f"Time taken: {(time.time() - start_time) / 60:.2f} minutes")

# Extract ViT features
extract_vit_features(train_loader, 'train')
extract_vit_features(val_loader, 'val')
extract_vit_features(test_loader, 'test')

# Verify ViT features
for split in ['train', 'val', 'test']:
    file_path = os.path.join(vit_feature_dir, f'{split}_vit_features.pt')
    if os.path.exists(file_path):
        data = torch.load(file_path, weights_only=False)
        print(f"{split} features shape: {data['features'].shape}")
        print(f"Sample caption: {data['captions'][0]}")
        print(f"Sample image name: {data['image_names'][0]}")
    else:
        print(f"{split}_vit_features.pt not found")

# Preprocess and save captions
df = pd.read_csv(annotation_file, sep=',', names=['image', 'caption'], skiprows=1)
preprocessed_captions = preprocess_caption(df['caption'].tolist())
pickle.dump(preprocessed_captions, open(r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\preprocessed_captions1.pkl', 'wb'))

# Vocabulary class
class Vocabulary:
    def __init__(self, min_freq=2):
        self.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>'}
        self.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
        self.min_freq = min_freq

    def build_vocabulary(self, captions):
        words = [word for caption_list in captions.values() for caption in caption_list for word in caption]
        word_counts = Counter(words)
        idx = len(self.itos)
        for word, count in word_counts.items():
            if count >= self.min_freq and word not in self.stoi:
                self.itos[idx] = word
                self.stoi[word] = idx
                idx += 1

    def numericalize(self, caption):
        return [self.stoi.get(word, self.stoi['<UNK>']) for word in caption]

# Caption dataset
class Flickr30kCaptionDataset(Dataset):
    def __init__(self, feature_file, caption_file, augment=False):
        data = torch.load(feature_file, weights_only=False)
        self.features = data['features']
        self.image_names = data['image_names']
        self.captions_dict = pickle.load(open(caption_file, 'rb'))
        self.augment = augment
        self.vocab = Vocabulary(min_freq=2)
        self.vocab.build_vocabulary(self.captions_dict)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        image_name = self.image_names[idx]
        captions = self.captions_dict.get(image_name, [[]])
        caption = captions[torch.randint(0, len(captions), (1,)).item()] if self.augment and len(captions) > 1 else captions[0]
        numerical_caption = [self.vocab.stoi['<SOS>']] + self.vocab.numericalize(caption) + [self.vocab.stoi['<EOS>']]
        return feature, torch.tensor(numerical_caption, dtype=torch.long), image_name

# Transformer model
class ImageCaptionTransformer(nn.Module):
    def __init__(self, feature_dim, vocab_size, embed_dim=512, num_heads=8, num_encoder_layers=6, num_decoder_layers=6,
                 hidden_dim=2048, dropout=0.05, max_len=100):
        super(ImageCaptionTransformer, self).__init__()
        self.feature_fc = nn.Linear(feature_dim, embed_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_len, embed_dim))
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim,
                                                   dropout=dropout, batch_first=True)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim,
                                                   dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        self.fc = nn.Linear(embed_dim, vocab_size)
        self.embed_dim = embed_dim
        self.max_len = max_len
        self.dropout = nn.Dropout(dropout)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.feature_fc.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.feature_fc.bias)
        nn.init.zeros_(self.fc.bias)

    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
        return mask

    def forward(self, features, captions=None, teacher_forcing_ratio=0.5):
        batch_size = features.size(0)
        device = features.device
        features = self.dropout(torch.relu(self.feature_fc(features)))
        features = features.unsqueeze(1)
        memory = self.transformer_encoder(features)
        if captions is None:
            return self.inference(features)
        captions = captions[:, :-1]
        embed = self.embedding(captions) * (self.embed_dim ** 0.5)
        positions = torch.arange(0, captions.size(1), device=device).unsqueeze(0).repeat(batch_size, 1)
        pos_embed = self.pos_encoder[:, :captions.size(1), :]
        embed = embed + pos_embed
        tgt_mask = self.generate_square_subsequent_mask(captions.size(1)).to(device)
        output = self.transformer_decoder(embed, memory, tgt_mask=tgt_mask)
        output = self.fc(output)
        return output

    def inference(self, features, max_len=20, beam_size=3):
        batch_size = features.size(0)
        device = features.device
        features = self.dropout(torch.relu(self.feature_fc(features)))
        features = features.unsqueeze(1)
        memory = self.transformer_encoder(features)
        sequences = [[torch.full((1, 1), 1, dtype=torch.long, device=device), 0.0]] * batch_size
        for _ in range(max_len):
            all_candidates = []
            for i in range(batch_size):
                candidates = []
                for seq, score in sequences[i]:
                    if seq[0, -1].item() == 2:
                        candidates.append([seq, score])
                        continue
                    embed = self.embedding(seq) * (self.embed_dim ** 0.5)
                    positions = torch.arange(0, seq.size(1), device=device).unsqueeze(0)
                    pos_embed = self.pos_encoder[:, :seq.size(1), :]
                    embed = embed + pos_embed
                    tgt_mask = self.generate_square_subsequent_mask(seq.size(1)).to(device)
                    output = self.transformer_decoder(embed, memory[i:i + 1], tgt_mask=tgt_mask)
                    output = self.fc(output[:, -1, :])
                    probs = torch.softmax(output, dim=-1)
                    top_probs, top_idx = probs.topk(beam_size, dim=-1)
                    for k in range(beam_size):
                        next_token = top_idx[0, k].unsqueeze(0).unsqueeze(0)
                        next_score = score - torch.log(top_probs[0, k]).item()
                        new_seq = torch.cat([seq, next_token], dim=1)
                        candidates.append([new_seq, next_score])
                candidates = sorted(candidates, key=lambda x: x[1])[:beam_size]
                all_candidates.append(candidates)
            sequences = all_candidates
        outputs = [sequences[i][0][0] for i in range(batch_size)]
        return torch.cat(outputs, dim=0)

# Load original captions
def load_original_captions(caption_txt_file):
    caption_dict = {}
    try:
        with open(caption_txt_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()[1:]  # Skip header
            for line in lines:
                parts = line.strip().split(',', 1)
                if len(parts) == 2:
                    image_name, caption = parts
                    caption_dict[image_name] = caption.split()
    except UnicodeDecodeError:
        with open(caption_txt_file, 'r', encoding='latin1') as f:
            lines = f.readlines()[1:]  # Skip header
            for line in lines:
                parts = line.strip().split(',', 1)
                if len(parts) == 2:
                    image_name, caption = parts
                    caption_dict[image_name] = caption.split()
    return caption_dict

# Evaluation function
def evaluate_model(model, loader, vocab, caption_dict, max_len=20, num_samples=5, beam_size=3):
    model.eval()
    references, hypotheses = [], []
    sample_outputs = []

    with torch.no_grad():
        for batch_idx, (features, captions, image_names) in enumerate(tqdm(loader, desc="Evaluating")):
            if features is None:
                continue
            features, captions = features.cuda(), captions.cuda()
            outputs = model.inference(features, max_len=max_len, beam_size=beam_size)
            for i in range(captions.size(0)):
                image_name = image_names[i]
                ref = caption_dict.get(image_name,
                                       [vocab.itos[idx.item()] for idx in captions[i] if idx.item() not in [0, 1, 2]])
                hyp = [vocab.itos[idx.item()] for idx in outputs[i] if idx.item() not in [0, 1, 2]]
                references.append([ref])
                hypotheses.append(hyp)
                if len(sample_outputs) < num_samples and batch_idx * captions.size(0) + i < num_samples:
                    sample_outputs.append({
                        'image_name': image_name,
                        'ground_truth': ' '.join(ref),
                        'generated': ' '.join(hyp)
                    })
    smoothing = SmoothingFunction().method1
    bleu_score = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)
    return bleu_score, sample_outputs

# Paths
feature_dir = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\ViT_Features'
caption_file = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\preprocessed_captions1.pkl'
caption_txt_file = r'D:\Projects\AIniverse\PROJECTS\mainstream\Scene_Description_Generator\flickr30k\versions\1\captions.txt'

# Datasets and DataLoaders
train_dataset = Flickr30kCaptionDataset(os.path.join(feature_dir, 'train_vit_features.pt'), caption_file, augment=True)
val_dataset = Flickr30kCaptionDataset(os.path.join(feature_dir, 'val_vit_features.pt'), caption_file, augment=False)
test_dataset = Flickr30kCaptionDataset(os.path.join(feature_dir, 'test_vit_features.pt'), caption_file, augment=False)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)

# Save vocab
with open(os.path.join(feature_dir, 'vocab.pkl'), 'wb') as f:
    pickle.dump(train_dataset.vocab, f)
print(f"Saved vocab.pkl with {len(train_dataset.vocab.itos)} tokens")

# Model
model = ImageCaptionTransformer(
    feature_dim=768,
    vocab_size=len(train_dataset.vocab.itos),
    embed_dim=512,
    num_heads=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    hidden_dim=2048,
    dropout=0.05,
    max_len=100
).cuda()

# Training setup
criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)
num_epochs = 100

# Training loop
def train_epoch(model, loader, criterion, optimizer, teacher_forcing_ratio=0.5):
    model.train()
    total_loss = 0
    for features, captions, _ in tqdm(loader, desc="Training"):
        if features is None:
            continue
        features, captions = features.cuda(), captions.cuda()
        outputs = model(features, captions, teacher_forcing_ratio)
        loss = criterion(outputs.view(-1, outputs.size(-1)), captions[:, 1:].contiguous().view(-1))
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Validation loop
def validate_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for features, captions, _ in tqdm(loader, desc="Validating"):
            if features is None:
                continue
            features, captions = features.cuda(), captions.cuda()
            outputs = model(features, captions, teacher_forcing_ratio=1.0)
            loss = criterion(outputs.view(-1, outputs.size(-1)), captions[:, 1:].contiguous().view(-1))
            total_loss += loss.item()
    return total_loss / len(loader)

# Training with early stopping
best_val_loss = float('inf')
patience = 3
no_improve_count = 0
min_delta = 0.01
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss = train_epoch(model, train_loader, criterion, optimizer, teacher_forcing_ratio=0.5)
    val_loss = validate_epoch(model, val_loader, criterion)
    scheduler.step()
    print(
        f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {(time.time() - start_time) / 60:.2f} min, LR: {optimizer.param_groups[0]['lr']:.6f}")
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        no_improve_count = 0
        torch.save(model.state_dict(), os.path.join(feature_dir, 'best_transformer_model.pt'))
        print(f"Saved best model with Val Loss: {best_val_loss:.4f}")
    else:
        no_improve_count += 1
        print(f"No improvement in Val Loss, count: {no_improve_count}/{patience}")
    if no_improve_count >= patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break